In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.ticker import MultipleLocator
import panel as pn
import hvplot.pandas
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

In [2]:
pn.extension(design='material', theme='default')

In [78]:
df = pd.read_csv('jobs_cleaned_1.csv')

In [4]:
qualifications = pd.read_csv('qualifications.csv')

In [11]:
del(qualifications['Unnamed: 0'])
qualifications = qualifications.rename(columns={'0': 'Qualifications'})
qualifications

,Qualifications
0,IT
1,MCP
2,COMPUTERSCIENCE
3,MCITP
4,2YEARS
...,...
13155,GIT
13156,COMMUNICATIONSKILLS
13157,SOFTWAREENGINEERING
13158,SVN


In [6]:
Type = pd.read_csv('type.csv')

In [10]:
del(Type['Unnamed: 0'])
Type = Type.rename(columns={'0': 'Type'})
Type

,Type
0,Full-time
1,Full-time
2,Full-time
3,Full-time
4,Full-time
...,...
745,Full-time
746,Full-time
747,Full-time
748,Full-time


In [8]:
benefits = pd.read_csv('benefits.csv')

In [9]:
del(benefits['Unnamed: 0'])
benefits = benefits.rename(columns={'0': 'Benefits'})
benefits

,Benefits
0,HEALTHINSURANCE
1,DENTALINSURANCE
2,LIFEINSURANCE
3,VISIONINSURANCE
4,HEALTHINSURANCE
...,...
4181,LIFEINSURANCE
4182,VISIONINSURANCE
4183,401(K)
4184,FLEXIBLESPENDINGACCOUNT


In [12]:
'''start_slider = pn.widgets.IntSlider(name='Start', start=0, end=934, value=0)
end_slider = pn.widgets.IntSlider(name='End', start=1, end=935, value=935)
column_selector = pn.widgets.Select(name='Column', options=list(df.columns), value='num_Qualifications')

def create_scatter(column, start, end):
    x=df[column][start:end].index
    y=df[column][start:end].values
    
    source = ColumnDataSource(data={'x': x, 'y': y})
    
    p = figure(title=f'Scatter plot of {column}', x_axis_label='Index', y_axis_label='Value', width=1000)
    p.scatter('x', 'y', source=source, size=9, color="navy", alpha=0.5)
    
    return p


pn.Column(
    column_selector,
    start_slider,
    end_slider,
    pn.panel(pn.bind(create_scatter, column_selector, start_slider, end_slider))
).servable()'''

'start_slider = pn.widgets.IntSlider(name=\'Start\', start=0, end=934, value=0)\nend_slider = pn.widgets.IntSlider(name=\'End\', start=1, end=935, value=935)\ncolumn_selector = pn.widgets.Select(name=\'Column\', options=list(df.columns), value=\'num_Qualifications\')\n\ndef create_scatter(column, start, end):\n    x=df[column][start:end].index\n    y=df[column][start:end].values\n    \n    source = ColumnDataSource(data={\'x\': x, \'y\': y})\n    \n    p = figure(title=f\'Scatter plot of {column}\', x_axis_label=\'Index\', y_axis_label=\'Value\', width=1000)\n    p.scatter(\'x\', \'y\', source=source, size=9, color="navy", alpha=0.5)\n    \n    return p\n\n\npn.Column(\n    column_selector,\n    start_slider,\n    end_slider,\n    pn.panel(pn.bind(create_scatter, column_selector, start_slider, end_slider))\n).servable()'

In [13]:
start_slider_scatter = pn.widgets.IntSlider(name='Start', start=0, end=934, value=0)
end_slider_scatter = pn.widgets.IntSlider(name='End', start=1, end=935, value=935)
column_selector_scatter = pn.widgets.Select(name='Column', options=['num_Qualifications', 'num_Benefits', 'Rating', 'Salary_Begin', 'Salary_End', 'Posted_Date', 'Posted_Month', 'Type', 'Job_simp', 'Salary_Average'], value='num_Qualifications')
color_scatter = pn.widgets.ColorPicker(value='navy')

def create_scatter(column, start, end, color):
    data = df[column][start:end]
    return data.reset_index().hvplot.scatter(x='index', y=column, height=500, width=600, color=color, size=20, alpha=.5, grid=True)

In [14]:
column_selector_hist = pn.widgets.Select(name='column', options=['Rating', 'num_Benefits', 'num_Qualifications', 'Salary_Begin', 'Salary_Average', 'Salary_End'], value='Rating')
color_hist = pn.widgets.ColorPicker(value='#FB93DA')

def create_hist(column, color):
    if column == 'num_Benefits' or column == 'num_Qualifications':
        data = df[column].apply(lambda x: round(x))
    else:
        data = df[column]
    return data.reset_index().hvplot.hist(column, color=color, height=532, width=600, grid=True)

In [106]:
import hvplot.xarray  # noqa
import xarray as xr

column_selector_bar = pn.widgets.Select(name='Column', options=['Job_simp', 'Company', 'Type', 'Qualifications', 'Benefits'], value='Company')

start_slider_bar = pn.widgets.IntSlider(name='start', start=0, end=10, value=0)
end_slider_bar = pn.widgets.IntSlider(name='end', start=1, end=31, value=20)

def set_range(event):
    if column_selector_bar.value=='Type':
        start_slider_bar.end=len(Type[column_selector_bar.value].value_counts())-1
        end_slider_bar.end=len(Type[column_selector_bar.value].value_counts())
    elif column_selector_bar.value=='Qualifications':
        start_slider_bar.end=len(qualifications[column_selector_bar.value].value_counts())-1
        end_slider_bar.end=len(qualifications[column_selector_bar.value].value_counts())
    elif column_selector_bar.value=='Benefits':
        start_slider_bar.end=len(benefits[column_selector_bar.value].value_counts())-1
        end_slider_bar.end=len(benefits[column_selector_bar.value].value_counts())
    else:
        start_slider_bar.end=len(df[column_selector_bar.value].value_counts())-1
        end_slider_bar.end=len(df[column_selector_bar.value].value_counts())

color_bar = pn.widgets.ColorPicker(value='blue')

column_selector_bar.param.watch(set_range, 'value')

def create_bar(column, color, start, end):
    if column=='Type':
        data=Type[column].value_counts()[start:end]
    elif column=='Qualifications':
        data=qualifications[column].value_counts()[start:end]
    elif column=='Benefits':
        data=benefits[column].value_counts()[start:end]
    else:
        data=df[column].value_counts()[start:end]
    
    return data.reset_index().hvplot.bar(x=column, rot=90, height=700, width=580, grid=True, color=color)

In [107]:
pn.Column(
    pn.Row(pn.Spacer(width=100), column_selector_scatter, color_scatter, pn.Spacer(width=160), column_selector_hist, color_hist),
    pn.Row(pn.Spacer(width=100), start_slider_scatter),
    pn.Row(pn.Spacer(width=100), end_slider_scatter),
    pn.Row(pn.panel(pn.bind(create_scatter, column_selector_scatter, start_slider_scatter, end_slider_scatter, color_scatter)), pn.panel(pn.bind(create_hist, column_selector_hist, color_hist))),
    pn.Row(pn.Spacer(width=100), column_selector_bar, color_bar),
    pn.Row(pn.Spacer(width=100), start_slider_bar),
    pn.Row(pn.Spacer(width=100), end_slider_bar),
    pn.Row(pn.Spacer(width=37), pn.panel(pn.bind(create_bar, column_selector_bar, color_bar, start_slider_bar, end_slider_bar)))
    
).servable()

Column(design=<class 'panel.theme.materi...)
    [0] Row(design=<class 'panel.theme.materi...)
        [0] Spacer(design=<class 'panel.theme.materi..., width=100)
        [1] Select(design=<class 'panel.theme.materi..., name='Column', options=['num_Qualifications', ...], value='Salary_End')
        [2] ColorPicker(design=<class 'panel.theme.materi..., value='navy')
        [3] Spacer(design=<class 'panel.theme.materi..., width=160)
        [4] Select(design=<class 'panel.theme.materi..., name='column', options=['Rating', 'num_Benefits',...], value='Rating')
        [5] ColorPicker(design=<class 'panel.theme.materi..., value='#FB93DA')
    [1] Row(design=<class 'panel.theme.materi...)
        [0] Spacer(design=<class 'panel.theme.materi..., width=100)
        [1] IntSlider(design=<class 'panel.theme.materi..., end=934, name='Start', value=315)
    [2] Row(design=<class 'panel.theme.materi...)
        [0] Spacer(design=<class 'panel.theme.materi..., width=100)
        [1] IntSlider(design=<class 'panel.theme.materi..., end=935, name='End', start=1, value=561)
    [3] Row(design=<class 'panel.theme.materi...)
        [0] ParamFunction(function, _pane=HoloViews, defer_load=False, design=<class 'panel.theme.materi...)
        [1] ParamFunction(function, _pane=HoloViews, defer_load=False, design=<class 'panel.theme.materi...)
    [4] Row(design=<class 'panel.theme.materi...)
        [0] Spacer(design=<class 'panel.theme.materi..., width=100)
        [1] Select(design=<class 'panel.theme.materi..., name='Column', options=['Job_simp', 'Company', ...], value='Company')
        [2] ColorPicker(design=<class 'panel.theme.materi..., value='blue')
    [5] Row(design=<class 'panel.theme.materi...)
        [0] Spacer(design=<class 'panel.theme.materi..., width=100)
        [1] IntSlider(design=<class 'panel.theme.materi..., end=10, name='start')
    [6] Row(design=<class 'panel.theme.materi...)
        [0] Spacer(design=<class 'panel.theme.materi..., width=100)
        [1] IntSlider(design=<class 'panel.theme.materi..., end=31, name='end', start=1, value=20)
    [7] Row(design=<class 'panel.theme.materi...)
        [0] Spacer(design=<class 'panel.theme.materi..., width=37)
        [1] ParamFunction(function, _pane=HoloViews, defer_load=False, design=<class 'panel.theme.materi...)